# Description

   * Quality control on PennRhiz v4bac library1 run1 post-merging and demultiplexing



# Setting variables

In [2]:
workDir = '/home/bryan/ERA/data/MiSeq/20170417_run1/QC/'
varSeqDir = '/home/bryan/ERA/data/MiSeq/20170417_run1/'
databaseDir = '/home/bryan/RhizCG/data/databases/'

seqFile = 'pear_merged-2017-04-18.assembled.dmult.fastq'
nprocs = 10
maxee = 2

# Init

In [2]:
from screed.fasta import fasta_iter
from pandas import DataFrame
import os
import re
import pandas as pd
from cogent import DNA
from qiime.assign_taxonomy import UclustConsensusTaxonAssigner
from IPython.display import Image


In [3]:
%load_ext rpy2.ipython

In [4]:
%%R
library(ggplot2)
library(dplyr)
library(tidyr)
library(gridExtra)

/opt/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 
Attaching package: ‘dplyr’


  warnings.warn(x, RRuntimeWarning)
/opt/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  warnings.warn(x, RRuntimeWarning)
/opt/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  warnings.warn(x, RRuntimeWarning)
/opt/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 
Attaching package: ‘gridExtra’


  warnings.warn(x, RRuntimeWarning)
/opt/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: The following object is masked from ‘package:dplyr’:

    combine


  warnings.warn(x, RRuntimeWarning)


In [5]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [6]:
if not os.path.isdir(workDir):
    os.mkdir(workDir)

In [7]:
!cd $workDir; ln -s -f ../$seqFile

# Merged read quality filtering

## Discard sequences that exceed max expected error threshold

In [8]:
%%bash -s "$workDir" "$seqFile" "$nprocs"

cd $1

mkdir tmp
mkdir tmp2


#max expected error
maxee=2


#tmpdir1= `mkdir tmp`
trap "rm -r tmp" 1 2 3  15
split -d -l 2000000 $1$2 tmp/Block



#tmpdir2=`mkdir tmp2`
trap "rm -r tmp2" 1 2 3 15
ls tmp/Block?? | parallel --gnu -j $3 -k "usearch -fastq_filter {} -fastq_maxee $maxee \
-fastaout tmp2/{#}.fasta >/dev/null 2>&1 && cat tmp2/{#}.fasta" > $1$2_maxee1.fasta
rm -r tmp tmp2

In [9]:
nlines = !cd $workDir; wc -l $seqFile
nlines = re.sub(" .+","", nlines[0])
nlines = int(nlines)
print "number of sequences pre-filter: {}".format(nlines/4)

maxee1 = seqFile + "_maxee1.fasta"
nlines2 = !cd $workDir; wc -l $maxee1
nlines2 = re.sub(" .+","", nlines2[0])
nlines2 = int(nlines2)
print "number of sequences post-filter: {}".format(nlines2/2)

number of sequences pre-filter: 14036217
number of sequences post-filter: 35084359


In [10]:
%%bash -s "$workDir" "$seqFile"  "$nprocs" "$maxee" 

printf "Max expected error cutoff:"
echo $4

printf "Number of sequence pre-filter: "
grep -c "orig_name" $1$2 

printf "Number of sequences post-filter: "
grep -c ">" $1$2_maxee1.fasta

echo '' 
#head -n 8 $1$2
head -n 8 $1$2_maxee1.fasta

Max expected error cutoff:2
Number of sequence pre-filter: 14036217
Number of sequences post-filter: 14034744

>ERA-T2_1-2a_85_0 orig_name=M02465:355:000000000-B3LCK:1:1101:15651:1332
TACGTAAGGGCCGAGCGTTGTCCGGAGTTACTGGGCGTAAAGCGCGCGCAGGCGGCTCGCTTTGCCCGGCGTGAAAGCCC
CCGGCTCAACCGGGGAGGGTCGTCGGGGACGGGCGAGCTTGAGGCCGGCAGGGGCAGGTGGAATTCCCGGTGTAGTGGTG
AAATGCGTAGAGATCGGGAGGAACACCCGTGGCGAAGGCGGCCTGCTGGGCCGGACCTGACGCTGAGGCGCGAAGGCGTG
GGGAGCGAACGGG
>ERA-T1_2-2d_85_1 orig_name=M02465:355:000000000-B3LCK:1:1101:15902:1335
TACAGAGGGTGCAAGCGTTGTTCGGAATCATTGGGCGTAAAGGGCGTGTAGGCGGTCTGCTAAGTCATGTGTGAAATCCC
TCGGCTCAACCGGGGAACGACGCATGAAACTGACAAGCTAGAGTACCAAAGAGGGGGGTGGAATTCCCGGTGTAGCGGTG


# Remove sequences with "N"

In [11]:
%%bash -s "$workDir" "$seqFile"

cd $1

bioawk -c fastx '{if ($seq !~ /N/){print ">" $name " " $4 "\n" $seq}}' $2_maxee1.fasta > $2_maxee1_noN.fasta

printf "Number of sequence pre-filter: "
grep -c ">" $2_maxee1.fasta

printf "Number of sequences post-filter: "
grep -c ">" $2_maxee1_noN.fasta

Number of sequence pre-filter: 14034744
Number of sequences post-filter: 14034743


## Alignment-based QC with Mothur

In [12]:
%%bash -s "$workDir" "$seqFile"

cd $1

mothur "#unique.seqs(fasta=$2_maxee1_noN.fasta)" 







mothur v.1.39.5
Last updated: 3/20/2017

by
Patrick D. Schloss

Department of Microbiology & Immunology
University of Michigan
http://www.mothur.org

When using, please cite:
Schloss, P.D., et al., Introducing mothur: Open-source, platform-independent, community-supported software for describing and comparing microbial communities. Appl Environ Microbiol, 2009. 75(23):7537-41.

Distributed under the GNU General Public License

Type 'help()' for information on the commands that are available

For questions and analysis support, please visit our forum at https://www.mothur.org/forum

Type 'quit()' to exit program



mothur > unique.seqs(fasta=pear_merged-2017-04-18.assembled.dmult.fastq_maxee1_noN.fasta)
1000	831
2000	1610
3000	2364
4000	3087
5000	3793
6000	4505
7000	5223
8000	5930
9000	6640
10000	7339
11000	8024
12000	8700
13000	9363
14000	10019
15000	10690
16000	11323
17000	11956
18000	12602
19000	13235
20000	13884
21000	14518
22000	15155
23000	15802
24000	16421
25000	17076
26000

In [13]:
%%bash -s "$workDir" "$seqFile"

cd $1

printf "Number of unique sequences: "
grep -c ">" $2_maxee1_noN.unique.fasta

Number of unique sequences: 5881633


In [14]:
%%bash -s "$workDir" "$seqFile"
# making mothur group file

cd $1

perl -ne 'if(/^>/){ s/>(.+)(_\d+) .+/$1$2\t$1/; print;}' $2_maxee1_noN.fasta > group_file.txt

head group_file.txt

ERA-T2_1-2a_85_0	ERA-T2_1-2a_85
ERA-T1_2-2d_85_1	ERA-T1_2-2d_85
ERA-T3_4-3d_170_2	ERA-T3_4-3d_170
ERA-T3_2-2c_170_3	ERA-T3_2-2c_170
ERA-T3_4-2c_85_4	ERA-T3_4-2c_85
ERA-T2_3-1b_85_5	ERA-T2_3-1b_85
MockComm_6	MockComm
ERA-T2_4-4a_85_7	ERA-T2_4-4a_85
ERA-T3_2-3d_170_8	ERA-T3_2-3d_170
ERA-T1_2-3b_85_9	ERA-T1_2-3b_85


In [15]:
# %%bash -s "$workDir"
# # dowloading mothur silva databases

# cd $1

# if ! [ -d mothur_silva_db ]; then
#     mkdir mothur_silva_db
# fi

# cd mothur_silva_db

# if ! [ -e $1silva_ref_aln_mothur.fasta ]; then
#     curl -o silva_Bac.zip http://www.mothur.org/w/images/9/98/Silva.bacteria.zip && unzip silva_Bac.zip
#     curl -o silva_Euk.zip http://www.mothur.org/w/images/1/1a/Silva.eukarya.zip && unzip silva_Euk.zip
#     curl -o silva_Arc.zip http://www.mothur.org/w/images/3/3c/Silva.archaea.zip && unzip silva_Arc.zip
# fi

In [16]:
!cd $workDir; ln -s -f $databaseDir

In [17]:
%%bash -s "$workDir"

cd $1'databases/mothur_silva_db'

cat silva.bacteria/silva.bacteria.fasta \
    silva.eukarya/silva.eukarya.fasta \
    Silva.archaea/silva.archaea.fasta \
    > silva_ref_aln_mothur.fasta

printf "Number of sequences in mothur silva fasta: "
grep -c ">" silva_ref_aln_mothur.fasta

Number of sequences in mothur silva fasta: 18491


In [ ]:
%%bash -s "$workDir"
# filtering column positions in silva db

cd $1'databases/mothur_silva_db'

mothur "#filter.seqs(vertical=t, \
    fasta=silva_ref_aln_mothur.fasta, \
    processors=24)" > /dev/null

printf "Number of sequences post-filter: "
grep -c ">" silva_ref_aln_mothur.filter.fasta

Number of sequences post-filter: 18491


In [ ]:
%%bash -s "$workDir" "$seqFile"
# aligning sequences

cd $1

mothur "#align.seqs(candidate=pear_merged\-2017\-04\-18.assembled.dmult.fastq_maxee1_noN.unique.fasta, \
    template=databases/mothur_silva_db/silva_ref_aln_mothur.filter.fasta, \
    processors=24, \
    flip=T)" | head -n 100

In [ ]:
%%bash -s "$workDir" "$seqFile"
# filtering out gap positions in the alignment

cd $1

mothur "#filter.seqs(vertical=t, \
    fasta=pear_merged\-2017\-04\-18.assembled.dmult.fastq_maxee1_noN.unique.align, \
    processors=12)" | head -n 50

In [ ]:
%%bash -s "$workDir" "$seqFile"
# filtering out gap positions in the alignment

cd $1

mothur "#summary.seqs(fasta=pear_merged-2017-04-18.assembled.dmult.fastq_maxee1_noN.unique.filter.fasta, \
    processors=24, \
    name=pear_merged-2017-04-18.assembled.dmult.fastq_maxee1_noN.names)" 

## Removing homopolymers (> 8) and screening out sequences that don't align to an amplicon region

In [ ]:
#continue here after checking alignment

In [ ]:
%%bash -s "$workDir" "$seqFile"

cd $1

mothur "#screen.seqs(fasta=pear_merged-2017-04-18.assembled.dmult.fastq_maxee1_noN.unique.filter.fasta, \
    processors=12, \
    name=pear_merged-2017-04-18.assembled.dmult.fastq_maxee1_noN.names, \
    group=group_file.txt, \
    start=252, \
    end=1163, \
    maxhomop=8, \
    minlength=50)" | tail -n 50   

In [ ]:
%%bash -s "$workDir" "$seqFile"

cd $1

printf "Number of sequences post-filter: "
grep -c ">" pear_merged-2017-04-18.assembled.dmult.fastq_maxee1_noN.unique.filter.good.fasta

#### Notes:

In [ ]:
%%bash -s "$workDir"

cd $1

mothur "#filter.seqs(fasta=pear_merged\-2017\-04\-18.assembled.dmult.fastq_maxee1_noN.unique.filter.good.fasta, \
    processors=12, \
    vertical=T)" | head -n 50

In [ ]:
%%bash -s "$workDir" 

cd $1

mothur "#summary.seqs(fasta=pear_merged-2017-04-18.assembled.dmult.fastq_maxee1_noN.unique.filter.good.fasta, \
    name=pear_merged-2017-04-18.assembled.dmult.fastq_maxee1_noN.good.names, processors=12)"

__Notes:__

* Not using reference-based chimera checking in Mothur with Uchime
   

## Deunique seqs

In [ ]:
%%bash -s "$workDir"

cd $1

mothur "#deunique.seqs(fasta=pear_merged-2017-04-18.assembled.dmult.fastq_maxee1_noN.unique.filter.good.fasta, \
    name=pear_merged-2017-04-18.assembled.dmult.fastq_maxee1_noN.good.names)" 

### Final QC-ed file

In [ ]:
!cd $workDir; \
    perl -pe 's/[-.]//g if ! /^>/' \
    pear_merged-2017-04-18.assembled.dmult.fastq_maxee1_noN.unique.filter.good.redundant.fasta \
    > finalQC.fasta

In [ ]:
!cd $workDir; \
    head finalQC.fasta

In [ ]:
!cd $workDir; \
    mothur "#summary.seqs(fasta=finalQC.fasta, processors=12)"

## Summary of number of seqs per sample

In [ ]:
os.chdir(workDir)

inFile = 'finalQC.summary'

re1 = re.compile('_\d+.+')

samp_count = dict()
with open(inFile, 'r') as inf:
    for line in inf.readlines():
        if line.startswith('seqname'):
            continue
        line = line.rstrip().split('\t')
        line[0] = re.sub(re1, '', line[0])
        try:
            samp_count[line[0]] += 1
        except KeyError:
            samp_count[line[0]] = 1

In [3]:
!cd $workDir; \
    head finalQC.summary



seqname	start	end	nbases	ambigs	polymer	numSeqs
ERA-T2_1-2a_85_0	1	253	253	0	5	1
ERA-T2_3-3a_85_4356060	1	253	253	0	5	1
ERA-T3_3-1c_0_9817383	1	253	253	0	5	1
ERA-T1_2-2d_85_1	1	253	253	0	6	1
ERA-T1_3-3a_170_223529	1	253	253	0	6	1
ERA-T1_3-3a_170_223945	1	253	253	0	6	1
ERA-T2_2-1c_85_450563	1	253	253	0	6	1
ERA-T1_3-3a_0_592632	1	253	253	0	6	1
ERA-T3_1-2b_0_683789	1	253	253	0	6	1


In [ ]:
# converting to dataframe
df_seq_cnt = pd.DataFrame(samp_count.items(), columns=['Sample', 'seq_count'])
df_seq_cnt


In [ ]:
#Write sample sums to file
df_seq_cnt.to_csv("run1_seq_cnt.csv")


In [ ]:
%%R -i df_seq_cnt -w 800
# plotting all

df_seq_cnt$seq_count = as.numeric(df_seq_cnt$seq_count)

df_seq_cnt = df_seq_cnt %>% 
    mutate(rank = min_rank(seq_count)) %>%
    arrange(desc(rank))

df_seq_cnt$Sample = factor(df_seq_cnt$Sample, levels=df_seq_cnt$Sample)

ggplot(df_seq_cnt, aes(Sample, seq_count)) +
    geom_bar(stat='identity') +
    theme(
        axis.text.x = element_text(angle=90)
    )

## Samples with a very low number of reads

In [38]:
%%R
df_seq_cnt %>% filter(seq_count < 100)

            Sample seq_count rank
1     PosControl_B        78    5
2     NegControl_C        75    4
3 PostiveControl_C        45    3
4      IndexQC_Rev         1    1
5       MockComm_6         1    1


In [39]:
%%R
df_seq_cnt %>% head

        Sample seq_count rank
1       ERA-T3   4829762   14
2       ERA-T1   4601591   13
3       ERA-T2   3750279   12
4       ERA-T0    660197   11
5     MockComm    118453   10
6 NegControl_B      1432    9
